In [1]:
import data_utils as du
from typing import Optional, List
import pandas as pd
from datetime import datetime, timedelta

In [2]:
hard_sales_df = du.load_hard_sales()
du.get_hw_names(hard_sales_df)

hard_sales_df = du.extract_by_hw(hard_sales_df, ['NSW', 'NS2', 'PS5', 'PS4', "XSX"])

# hard_sales_dfの最後から20行程度を表示する
hard_sales_df.tail(20)


,weekly_id,begin_date,end_date,report_date,period_date,hw,units,year,month,mday,week,delta_day,delta_week,delta_year,avg_units,sum_units,launch_date,maker_name,full_name
8188,2025-06-29_NSW,2025-06-23,2025-06-29,2025-06-29,7,NSW,15323,2025,6,29,26,3040,434,8,2189,35793449,2017-03-03,Nintendo,Nintendo Switch
8189,2025-06-29_NS2,2025-06-23,2025-06-29,2025-06-29,7,NS2,161021,2025,6,29,26,24,3,0,23003,1402183,2025-06-05,Nintendo,Nintendo Switch2
8190,2025-06-29_PS4,2025-06-23,2025-06-29,2025-06-29,7,PS4,23,2025,6,29,26,4145,592,11,3,9490597,2014-02-22,SONY,PlayStation4
8191,2025-06-29_PS5,2025-06-23,2025-06-29,2025-06-29,7,PS5,7333,2025,6,29,26,1690,241,5,1047,6910279,2020-11-12,SONY,PlayStation5
8192,2025-06-29_XSX,2025-06-23,2025-06-29,2025-06-29,7,XSX,353,2025,6,29,26,1692,241,5,50,680598,2020-11-10,Microsoft,Xbox Series X|S
8193,2025-07-06_NSW,2025-06-30,2025-07-06,2025-07-06,7,NSW,17712,2025,7,6,27,3047,435,8,2530,35811161,2017-03-03,Nintendo,Nintendo Switch
8194,2025-07-06_NS2,2025-06-30,2025-07-06,2025-07-06,7,NS2,128643,2025,7,6,27,31,4,0,18377,1530826,2025-06-05,Nintendo,Nintendo Switch2
8195,2025-07-06_PS4,2025-06-30,2025-07-06,2025-07-06,7,PS4,21,2025,7,6,27,4152,593,11,3,9490618,2014-02-22,SONY,PlayStation4
8196,2025-07-06_PS5,2025-06-30,2025-07-06,2025-07-06,7,PS5,8629,2025,7,6,27,1697,242,5,1232,6918908,2020-11-12,SONY,PlayStation5
8197,2025-07-06_XSX,2025-06-30,2025-07-06,2025-07-06,7,XSX,529,2025,7,6,27,1699,242,5,75,681127,2020-11-10,Microsoft,Xbox Series X|S


In [5]:
latest_df = du.extract_latest(hard_sales_df)
latest_df   

,weekly_id,begin_date,end_date,report_date,period_date,hw,units,year,month,mday,week,delta_day,delta_week,delta_year,avg_units,sum_units,launch_date,maker_name,full_name
8203,2025-07-20_NSW,2025-07-14,2025-07-20,2025-07-20,7,NSW,11766,2025,7,20,29,3061,437,8,1680,35836338,2017-03-03,Nintendo,Nintendo Switch
8204,2025-07-20_NS2,2025-07-14,2025-07-20,2025-07-20,7,NS2,152165,2025,7,20,29,45,6,0,21737,1754876,2025-06-05,Nintendo,Nintendo Switch2
8205,2025-07-20_PS4,2025-07-14,2025-07-20,2025-07-20,7,PS4,22,2025,7,20,29,4166,595,11,3,9490670,2014-02-22,SONY,PlayStation4
8206,2025-07-20_PS5,2025-07-14,2025-07-20,2025-07-20,7,PS5,6148,2025,7,20,29,1711,244,5,878,6932318,2020-11-12,SONY,PlayStation5
8207,2025-07-20_XSX,2025-07-14,2025-07-20,2025-07-20,7,XSX,150,2025,7,20,29,1713,244,5,21,681524,2020-11-10,Microsoft,Xbox Series X|S


In [6]:
filtered_df = du.extract_by_date(hard_sales_df, '2025-06-05')
filtered_df

,weekly_id,begin_date,end_date,report_date,period_date,hw,units,year,month,mday,week,delta_day,delta_week,delta_year,avg_units,sum_units,launch_date,maker_name,full_name
8173,2025-06-08_NSW,2025-06-02,2025-06-08,2025-06-08,7,NSW,16611,2025,6,8,23,3019,431,8,2373,35752038,2017-03-03,Nintendo,Nintendo Switch
8174,2025-06-08_NS2,2025-06-02,2025-06-08,2025-06-08,7,NS2,947931,2025,6,8,23,3,0,0,135418,947931,2025-06-05,Nintendo,Nintendo Switch2
8175,2025-06-08_PS4,2025-06-02,2025-06-08,2025-06-08,7,PS4,24,2025,6,8,23,4124,589,11,3,9490524,2014-02-22,SONY,PlayStation4
8176,2025-06-08_PS5,2025-06-02,2025-06-08,2025-06-08,7,PS5,20782,2025,6,8,23,1669,238,5,2968,6882811,2020-11-12,SONY,PlayStation5
8177,2025-06-08_XSX,2025-06-02,2025-06-08,2025-06-08,7,XSX,333,2025,6,8,23,1671,238,5,47,679793,2020-11-10,Microsoft,Xbox Series X|S


In [8]:
df1M = du.extract_week_reached_units(hard_sales_df, 1500000)
df1M

,weekly_id,begin_date,end_date,report_date,period_date,hw,units,year,month,mday,week,delta_day,delta_week,delta_year,avg_units,sum_units,launch_date,maker_name,full_name
8194,2025-07-06_NS2,2025-06-30,2025-07-06,2025-07-06,7,NS2,128643,2025,7,6,27,31,4,0,18377,1530826,2025-06-05,Nintendo,Nintendo Switch2
6285,2017-08-27_NSW,2017-08-21,2017-08-27,2017-08-27,7,NSW,85503,2017,8,27,34,177,25,0,12214,1565273,2017-03-03,Nintendo,Nintendo Switch
5701,2015-08-02_PS4,2015-07-27,2015-08-02,2015-08-02,7,PS4,13192,2015,8,2,31,526,75,1,1884,1508068,2014-02-22,SONY,PlayStation4
7444,2022-04-03_PS5,2022-03-28,2022-04-03,2022-04-03,7,PS5,30666,2022,4,3,13,507,72,2,4380,1528183,2020-11-12,SONY,PlayStation5


In [5]:
switch_df = du.extract_by_hw(hard_sales_df, ['PS5', 'NS2'])

# Nintendo Switch* の最後から20行程度を表示する
switch_df.tail(20)


,weekly_id,begin_date,end_date,report_date,period_date,hw,units,year,month,mday,week,delta_day,delta_week,delta_year,avg_units,sum_units,launch_date,maker_name,full_name
8151,2025-04-20_PS5,2025-04-14,2025-04-20,2025-04-20,7,PS5,7497,2025,4,20,16,1620,231,5,1071,6804660,2020-11-12,SONY,PlayStation5
8155,2025-05-04_PS5,2025-04-21,2025-05-04,2025-05-04,14,PS5,20204,2025,5,4,18,1634,233,5,1443,6824864,2020-11-12,SONY,PlayStation5
8159,2025-05-11_PS5,2025-05-05,2025-05-11,2025-05-11,7,PS5,8615,2025,5,11,19,1641,234,5,1230,6833479,2020-11-12,SONY,PlayStation5
8163,2025-05-18_PS5,2025-05-12,2025-05-18,2025-05-18,7,PS5,6546,2025,5,18,20,1648,235,5,935,6840025,2020-11-12,SONY,PlayStation5
8167,2025-05-25_PS5,2025-05-19,2025-05-25,2025-05-25,7,PS5,6738,2025,5,25,21,1655,236,5,962,6846763,2020-11-12,SONY,PlayStation5
8171,2025-06-01_PS5,2025-05-26,2025-06-01,2025-06-01,7,PS5,15266,2025,6,1,22,1662,237,5,2180,6862029,2020-11-12,SONY,PlayStation5
8174,2025-06-08_NS2,2025-06-02,2025-06-08,2025-06-08,7,NS2,947931,2025,6,8,23,3,0,0,135418,947931,2025-06-05,Nintendo,Nintendo Switch2
8176,2025-06-08_PS5,2025-06-02,2025-06-08,2025-06-08,7,PS5,20782,2025,6,8,23,1669,238,5,2968,6882811,2020-11-12,SONY,PlayStation5
8179,2025-06-15_NS2,2025-06-09,2025-06-15,2025-06-15,7,NS2,153205,2025,6,15,24,10,1,0,21886,1101136,2025-06-05,Nintendo,Nintendo Switch2
8181,2025-06-15_PS5,2025-06-09,2025-06-15,2025-06-15,7,PS5,14129,2025,6,15,24,1676,239,5,2018,6896940,2020-11-12,SONY,PlayStation5


In [11]:
# 昨年のデータ
last_year_df = du.extract_by_year(hard_sales_df, 2024)
last_year_df

,weekly_id,begin_date,end_date,report_date,period_date,hw,units,year,month,mday,week,delta_day,delta_week,delta_year,avg_units,sum_units,launch_date,maker_name,full_name
7876,2024-01-07_XSX,2024-01-01,2024-01-07,2024-01-07,7,XSX,1324,2024,1,7,1,1153,164,4,189,541431,2020-11-10,Microsoft,Xbox Series X|S
7878,2024-01-07_NSW,2024-01-01,2024-01-07,2024-01-07,7,NSW,104551,2024,1,7,1,2501,357,7,14935,31891225,2017-03-03,Nintendo,Nintendo Switch
7879,2024-01-07_PS4,2024-01-01,2024-01-07,2024-01-07,7,PS4,737,2024,1,7,1,3606,515,10,105,9476969,2014-02-22,SONY,PlayStation4
7880,2024-01-07_PS5,2024-01-01,2024-01-07,2024-01-07,7,PS5,41126,2024,1,7,1,1151,164,4,5875,5005983,2020-11-12,SONY,PlayStation5
7882,2024-01-14_NSW,2024-01-08,2024-01-14,2024-01-14,7,NSW,46604,2024,1,14,2,2508,358,7,6657,31937829,2017-03-03,Nintendo,Nintendo Switch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084,2024-12-22_XSX,2024-12-16,2024-12-22,2024-12-22,7,XSX,3389,2024,12,22,51,1503,214,4,484,657167,2020-11-10,Microsoft,Xbox Series X|S
8085,2024-12-29_NSW,2024-12-23,2024-12-29,2024-12-29,7,NSW,76548,2024,12,29,52,2858,408,7,10935,34895777,2017-03-03,Nintendo,Nintendo Switch
8086,2024-12-29_PS4,2024-12-23,2024-12-29,2024-12-29,7,PS4,37,2024,12,29,52,3963,566,10,5,9489960,2014-02-22,SONY,PlayStation4
8087,2024-12-29_PS5,2024-12-23,2024-12-29,2024-12-29,7,PS5,27130,2024,12,29,52,1508,215,4,3875,6419006,2020-11-12,SONY,PlayStation5


In [4]:
nintendo_df = du.extract_by_maker(hard_sales_df, ['Nintendo'])
nintendo_df.tail(20)

,weekly_id,begin_date,end_date,report_date,period_date,hw,units,year,month,mday,week,delta_day,delta_week,delta_year,avg_units,sum_units,launch_date,maker_name,full_name
8149,2025-04-20_NSW,2025-04-14,2025-04-20,2025-04-20,7,NSW,25408,2025,4,20,16,2970,424,8,3629,35580817,2017-03-03,Nintendo,Nintendo Switch
8153,2025-05-04_NSW,2025-04-21,2025-05-04,2025-05-04,14,NSW,56058,2025,5,4,18,2984,426,8,4004,35636875,2017-03-03,Nintendo,Nintendo Switch
8157,2025-05-11_NSW,2025-05-05,2025-05-11,2025-05-11,7,NSW,29710,2025,5,11,19,2991,427,8,4244,35666585,2017-03-03,Nintendo,Nintendo Switch
8161,2025-05-18_NSW,2025-05-12,2025-05-18,2025-05-18,7,NSW,24478,2025,5,18,20,2998,428,8,3496,35691063,2017-03-03,Nintendo,Nintendo Switch
8165,2025-05-25_NSW,2025-05-19,2025-05-25,2025-05-25,7,NSW,24289,2025,5,25,21,3005,429,8,3469,35715352,2017-03-03,Nintendo,Nintendo Switch
8169,2025-06-01_NSW,2025-05-26,2025-06-01,2025-06-01,7,NSW,20075,2025,6,1,22,3012,430,8,2867,35735427,2017-03-03,Nintendo,Nintendo Switch
8173,2025-06-08_NSW,2025-06-02,2025-06-08,2025-06-08,7,NSW,16611,2025,6,8,23,3019,431,8,2373,35752038,2017-03-03,Nintendo,Nintendo Switch
8174,2025-06-08_NS2,2025-06-02,2025-06-08,2025-06-08,7,NS2,947931,2025,6,8,23,3,0,0,135418,947931,2025-06-05,Nintendo,Nintendo Switch2
8178,2025-06-15_NSW,2025-06-09,2025-06-15,2025-06-15,7,NSW,13319,2025,6,15,24,3026,432,8,1902,35765357,2017-03-03,Nintendo,Nintendo Switch
8179,2025-06-15_NS2,2025-06-09,2025-06-15,2025-06-15,7,NS2,153205,2025,6,15,24,10,1,0,21886,1101136,2025-06-05,Nintendo,Nintendo Switch2
